<a href="https://colab.research.google.com/github/QwertyApoorv987/Object_Detection/blob/main/Project_Starting_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import LabelBinarizer # Label encoding, 1-hot encoding, multi-encoding
# LABEL binarizer is a 1-hot encoded MATRIX 
import cv2
import numpy as np
import random
import imutils
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from sklearn.model_selection import train_test_split
from keras import backend # backend -> tf/theano etc
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential # no parallel, no cycles 

In [2]:
HP_dataset = 'Data'
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'
HP_test_dataset = 'test'
HP_epoch = 100
HP_init_lr = 1e-3 # learning_rate = 0.001
HP_batch_size = 32
HP_image_dim = (96,96,3)

In [3]:
data = []
labels = [] 
# read all images
all_images = sorted(list(paths.list_images(HP_dataset))) ### listing all the images that have Data in their path
all_images[:5]

['Data/Cat/animal-image_0.png',
 'Data/Cat/animal-image_1.png',
 'Data/Cat/animal-image_10.png',
 'Data/Cat/animal-image_100.png',
 'Data/Cat/animal-image_101.png']

In [4]:
random.seed(42)
random.shuffle(all_images)

In [5]:
data = []
labels = []
import os
for impath in all_images:
  img = cv2.imread(impath)
  resized = cv2.resize(img, (HP_image_dim[0],HP_image_dim[1]) ) ### every image to (96,96) aspect ratio
  imageData = img_to_array(resized)
  data.append(imageData) #### data contains every image , data[0] first image
  # extract label from filename (2nd last element) / \\ 
  label = impath.split(os.path.sep)[-2]
  labels.append(label) ### labels containing Dogs, Jaguars corresponding to each image

In [ ]:
##### data contains images and labels contain labels

In [ ]:
####### My code starting 

In [ ]:
######################################################################################################

In [11]:
print(len(data))
print(len(labels))

450
450


In [7]:
labels = np.array(labels)
lb = LabelBinarizer()
print(labels)
binarized_labels = lb.fit_transform(labels)  
print(binarized_labels[1])  #### not working as of now but would work


################### also lb.classes_ contains the name of all the labels
###### binarised labels and data  ###################################

['Cat' 'Cat' 'Dog' 'Dog' 'Dog' 'Jaguar' 'Dog' 'Cat' 'Cat' 'Cat' 'Cat'
 'Jaguar' 'Dog' 'Dog' 'Dog' 'Dog' 'Jaguar' 'Dog' 'Dog' 'Cat' 'Jaguar'
 'Cat' 'Dog' 'Cat' 'Cat' 'Jaguar' 'Dog' 'Cat' 'Jaguar' 'Cat' 'Cat' 'Cat'
 'Cat' 'Cat' 'Jaguar' 'Jaguar' 'Dog' 'Dog' 'Jaguar' 'Dog' 'Dog' 'Cat'
 'Cat' 'Jaguar' 'Dog' 'Dog' 'Dog' 'Jaguar' 'Cat' 'Jaguar' 'Jaguar' 'Cat'
 'Dog' 'Jaguar' 'Cat' 'Cat' 'Dog' 'Dog' 'Jaguar' 'Jaguar' 'Cat' 'Dog'
 'Jaguar' 'Jaguar' 'Cat' 'Jaguar' 'Cat' 'Jaguar' 'Dog' 'Jaguar' 'Cat'
 'Cat' 'Cat' 'Dog' 'Cat' 'Jaguar' 'Dog' 'Jaguar' 'Jaguar' 'Jaguar'
 'Jaguar' 'Dog' 'Jaguar' 'Dog' 'Cat' 'Dog' 'Jaguar' 'Dog' 'Dog' 'Cat'
 'Dog' 'Jaguar' 'Dog' 'Dog' 'Cat' 'Dog' 'Jaguar' 'Dog' 'Cat' 'Dog' 'Cat'
 'Cat' 'Jaguar' 'Cat' 'Cat' 'Dog' 'Dog' 'Cat' 'Dog' 'Jaguar' 'Dog' 'Cat'
 'Cat' 'Dog' 'Dog' 'Jaguar' 'Cat' 'Cat' 'Cat' 'Dog' 'Dog' 'Dog' 'Jaguar'
 'Jaguar' 'Dog' 'Dog' 'Cat' 'Jaguar' 'Jaguar' 'Cat' 'Dog' 'Dog' 'Jaguar'
 'Dog' 'Dog' 'Cat' 'Jaguar' 'Dog' 'Dog' 'Cat' 'Cat' 'Cat' 'Dog' 'Cat'
 'Dog

In [25]:
print(lb.classes_)

['Cat' 'Dog' 'Jaguar']


In [8]:
norm_data = np.array(data, dtype='float') / 255 
trainx, testx, trainy, testy = train_test_split(norm_data,binarized_labels,test_size=0.2,random_state=42)
# train and test data ready 
# Augmentation generators are logic for augmentation
# APplication of augmentation is after this


####### normalizing and splitting into train and test 

In [22]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1,
                         shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
more_images = []
more_images= aug.flow_from_directory(
    directory=r"./Data/",
    target_size=(96,96),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)

Found 450 images belonging to 4 classes.


In [23]:
len(more_images)

15

In [26]:
class CorrectPandaVGG: # model.fit(), model.predict() ; for them you need object
  @staticmethod
  def create(width, height, depth, classes): # TF was CHANNELS_LAST -> depth is mentioned as the last value, after w&h
    model = Sequential()
    inputShape = (height, width, depth) # the expected shape as per TF
    # but what if this program is then run on some other framework? which follows channels_first instead?
    chanDim = -1 # CHANNELS_LAST set as default 
    # now check if some other library except TF or CNTK is running
    if backend.image_data_format() == "channels_first":
      inputShape = (depth, height, width) # this chan_dim is a measure of DEPTH of data-> so makes sense only for multidimension data, such as image or video
      chanDim = 1 # for Theano (if tf or CNTK is not running)
    # Design our network
    # FIRST BLOCK = Conv + ReLU + BN + MP + D
    model.add(Conv2D(32,(3,3), padding='same', input_shape=inputShape)) # all keras backends compatible
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim)) # Normalization needs to be along axis- R,G and B! 
    # After BN, the important FILTERED features are available now as WEIGHTS 
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25)) 
    # Learning Layers -> Conv + ReLU + BN
    # that means, to increase the number of features, i need to repeat the Conv+ReLU+BN to generate more and more
    # features
    # MAX Pool is to compress, and Dropout -> randomly select features for partial learning 
    
    # BLOCK 2 should have slightly more features, because we have compressed and DROPPED the data!!! 
    # we should generate more activity maps henceforth to compensate for data loss 
    # BLOCK 2 = (Conv->Relu->BN)X2 + MP + D
    # LET X = Conv->Relu->BN
    # Block 2 = 2 X + MP + D
    # My network so far = Block 1 + Block 2
    # FeatureClassification = Flatten + Dense(ReLU) + Dense(SoftMax)
    # PandaVGG = X + MP + D +   ( 2 X + MP + D ) * no_of_times_you_want_to_repeat + FeatureClassification
    # First X -> got activity map for 100% of data (X=Conv+Relu+BN)
    # then Activity Maps are compressed and sent to subsequent layers (MaxPooling)
    # Dropout to learn about only a section of data (Dropout)
    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2))) # SMALLER pool size-> finer features selected -> BIGGER compression
    model.add(Dropout(0.25))

    # block 3
    model.add(Conv2D(128,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(128,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2))) # SMALLER pool size-> more data lost -> BIGGER compression
    model.add(Dropout(0.25))
    # block 4
    model.add(Conv2D(256,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(256,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2))) # SMALLER pool size-> more data lost -> BIGGER compression
    model.add(Dropout(0.25))
    #VGG has now become = block1 +block2+block3+block4 + FeatureClassification
    # VGG= X(32)+MP(3,3)+D+2X(64)+MP(2,2)+D+2X(128)+MP(2,2)+D+2X(256)+MP(2,2)+D+Flatten+DenseRelu+DenseSoftMax
    # Where X = Conv2D + ReLU + BN 
    # Our PandaVGG now has 32 layers

    #FINAL Classification block
    # ALL micro and macro features (FILTERsX Input = Activity Map) have been generated
    # SO FLATTEN out to bring all axises into same DIMENSION
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Dense(512))
    model.add(Activation('relu'))
    # optional: EVEN more regularization
    #model.add(BatchNormalization())
    #model.add(Dropout(0.5))
    #regularization over, Classify using SOFTMAX


#################### here on needs work  #################################################
  
    model.add(Dense(classes))  
    model.add(Activation('softmax'))
    return model





###################### Main work needs to be done 1)what should be the final activation layer
##############################2) how to mark the box dimensions for training images

In [ ]:
#####################clear these 

##################################



In [28]:
from keras.optimizers import Adam
# or use the previous class, both are same
model = CorrectPandaVGG.create(HP_image_dim[0], HP_image_dim[1], HP_image_dim[2],len(lb.classes_))
optimizer = Adam(lr=HP_init_lr, decay= HP_init_lr/HP_epoch)


################# figure out loss and metrics ###########################
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [29]:
history = model.fit_generator(aug.flow(trainx, trainy, batch_size=HP_batch_size), 
                              validation_data=(testx,testy),
                              steps_per_epoch = len(trainx)/HP_batch_size,
                              epochs=HP_epoch)
####steps_per_epoch: 
### Total number of steps (batches of samples) to yield from generator 
##### before declaring one epoch finished and starting the next epoch.

#### So in every epoch (number of images in training set/ batch_size) these many times images are generated using aug

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
11/11 [==============================] - 14s 1s/step - loss: 12.6100 - accuracy: 0.3486 - val_loss: 1.2012 - val_accuracy: 0.3111
Epoch 2/100
11/11 [==============================] - 12s 1s/step - loss: 4.0923 - accuracy: 0.3646 - val_loss: 1.3726 - val_accuracy: 0.3000
Epoch 3/100
11/11 [==============================] - 12s 1s/step - loss: 4.0433 - accuracy: 0.4027 - val_loss: 1.3690 - val_accuracy: 0.3000
Epoch 4/100
11/11 [==============================] - 12s 1s/step - loss: 2.5290 - accuracy: 0.5492 - val_loss: 1.5479 - val_accuracy: 0.3000
Epoch 5/100
11/11 [==============================] - 12s 1s/step - loss: 2.0237 - accuracy: 0.5403 - val_loss: 1.1076 - val_accuracy: 0.2889
Epoch 6/100
11/11 [==============================] - 12s 1s/step - loss: 1.5809 - accuracy: 0.5410 - val_loss: 3.0124 - val_accuracy: 0.3000
Epoch 7/100
11/11 [==============================] - 12s 1s/step - loss: 1.6956 - accuracy: 0.4521 - val_loss: 2.4339 - val_accuracy: 0.2889
Epoch 8/100


In [30]:
HP_model_path = 'bin/model/myModel.h5'
HP_binarized_labels = 'bin/labels/labels.bin'
model.save(HP_model_path)   ####################### saving model ######################################

In [ ]:
f = open(HP_binarized_labels, 'wb') # our integer is represent in binary format!
f.write(pickle.dumps(lb))
f.close()


####
f = open(HP_binarized_labels, 'rb')
mylabels = f.read()
binarized_labels = pickle.loads(mylabels)


###### this cell is just storing the labels and then again constructing back with the help of file 


In [ ]:
#######################################    testing is left #################################################################